In [ ]:
using LinearAlgebra                                                             
using Test                                                                                       
using Distributions                                                             
using DataFrames                                                                          
using CSV                                                                              
using Polynomials                                                               
                                                                                 
include("../computation/finitedifference.jl")                                                   
include("../quadrature/quadrature.jl")                                             
include("../transforms/transforms.jl")                                             
include("../priors/priors.jl")                                                     
include("../bayesopt/incremental.jl")                                              
include("../kernels/kernel.jl")                                                    
include("../datastructs.jl")                                                       
include("../computation/buffers0.jl") #datastruct, kernel, incremental, quadrature 
include("../model0.jl") #buffers, datastructs, several auxiliary                   
include("../computation/tdist.jl") #model0 and buffer0  

In [16]:
test_fun1(x) = sin(x) + 1.1
x = reshape([0, 0.3, 0.5, 0.8, 1, 1.3, 1.6, 2], 8, 1)
Fx = ones(10, 1)
y = reshape(test_fun1.(x), 8)
trainingData1 = trainingData(x, Fx, y) #training data used for testing various functions

trainingData([0.0; 0.3; … ; 1.6; 2.0], [1.0; 1.0; … ; 1.0; 1.0], [1.1, 1.3955202066613397, 1.579425538604203, 1.8173560908995228, 1.9414709848078966, 2.063558185417193, 2.099573603041505, 2.0092974268256816], 1, 1, 8)

In [17]:
d = getDimension(trainingData1)
n = getNumPts(trainingData1)
p = getCovDimension(trainingData1)
rangeθ = [0.25 100]
rangeθ = repeat(rangeθ, d, 1)
rangeλ = [0.5 5]

1×2 Array{Float64,2}:
 0.5  5.0

In [18]:
btg1 = btg(trainingData1, rangeθ, rangeλ);

Iterating over nodeset to build train_buffer_dict...


DimensionMismatch: DimensionMismatch("first dimension of B, 10, and size of A, (8,8), must match!")

In [ ]:
(dpdf, pdf, cdf) = solve(btg1);